In [1]:
from regression_only import *

reg_models = load_regression_models()

In [2]:
df, ki_range = import_data()

x = df[df.columns[1:573]]
y = df['KI (nM) rescaled']

scaler = MinMaxScaler()
scaler.fit(x)
x = pd.DataFrame(scaler.transform(x), columns=df.columns[1:573])

In [3]:
x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size=(1/5), random_state=42)

In [6]:
x.shape[1]

572